In [34]:
import numpy as np 
import cv2
from ultralytics import YOLO
import torch
import math
# Load image
image_path = '/home/taheuber/datasets/Train-Tracks-2-1/test/images/WIN_20220602_13_39_06_Pro_jpg.rf.9d150ea5b5a0d862313c7f2af6e70f1f.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (640, 640))
c_x = 
c_y =
f_x =
f_y =
if image is None:
    print("Error: Unable to load image.")
    exit()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load YOLO model and get segmentation masks
model = YOLO("/home/taheuber/runs/segment/train4/weights/best.pt")
results = model(image, device='0', imgsz=640, conf=0.6, half=True)
boxes = results[0].boxes.data
masks = results[0].masks.data  # Get segmentation masksclss

clss = boxes[:,5]
track_indices = torch.where(clss == 0)
track_masks = masks[track_indices]
track_masks = torch.any(track_masks, dim=0).int() * 255
track_masks_uint8 = track_masks.cpu().numpy().astype(np.uint8)
# Create a binary mask from the segmentation masks
if masks is not None:
    # Convert masks to a binary representation and combine all masks into a single mask
 
    # Apply the binary mask to the original image
    masked_image = cv2.bitwise_and(image, image, mask=track_masks_uint8)  # Keep only masked areas

    # Apply Canny edge detection only to the masked area
    gray_masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_masked_image, 250, 300)  # Apply edge detection

    # Hough Line Transformation
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=10, maxLineGap=10)

    # Create a copy of the original image to draw detected lines 
    line_image = np.copy(image)
    # Determine the longest line
    longest_line = None
    max_length = 0
    # Draw the detected lines on the copy of the original image
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            length = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            if length > max_length:
                max_length = length
                longest_line = (x1, y1, x2, y2)
    x1, y1, x2, y2 = longest_line
    angle = math.atan2(y2 - y1, x2 - x1)  # In radians
    
    # Display the starting point and draw the longest line
    line_image = np.copy(image)
    cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Draw the line in red
    cv2.circle(line_image, (x2, y2), radius=5, color=(0, 255, 0), thickness=-1)  # Mark the start point in green
    
    cv2.imshow("Image with Longest Line and Start Point", line_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print("No masks found in the segmentation results.")



0: 640x640 1 tracks, 10.6ms
Speed: 1.8ms preprocess, 10.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


In [26]:
print("lines",lines)


lines [[[494 246 639 284]]

 [[471 242 631 290]]]
